<a href="https://colab.research.google.com/github/csetapan/MyResearchFile/blob/master/MobileNet_SVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import os
import cv2

In [ ]:
CATEGORIES = ['CT_COVID','CT_NonCOVID']
DATADIR = '/content/drive/MyDrive/Images-processed/train'
SIZE = 256
train_image = []
train_labels = []
for category in CATEGORIES:
    path=os.path.join(DATADIR,category) # paths to CT_COVID or CT_NonCOVID directory
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
        #img_array=cv2.resize(img_array,(SIZE,SIZE))
        try:
          img_array=cv2.resize(img_array,(SIZE,SIZE))
        except:
          break
        train_image.append(img_array)
        train_labels.append(category)

In [ ]:
CATEGORIES = ['CT_COVID','CT_NonCOVID']
DATADIR = '/content/drive/MyDrive/Images-processed/test'
test_image = []
test_labels = []
SIZE = 256
for category in CATEGORIES:
    path=os.path.join(DATADIR,category) # paths to CT_COVID or CT_NonCOVID directory
    for img in os.listdir(path):
        img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_COLOR)
        img_array=cv2.resize(img_array,(SIZE,SIZE))
        test_image.append(img_array)
        test_labels.append(category)

In [ ]:
#Convert lists to arrays                
test_images = np.array(test_image)
test_labels = np.array(test_labels)
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_image, train_labels_encoded, test_images, test_labels_encoded
#x_train,y_train = train_image,train_labels_encoded

In [ ]:
###################################################################
from sklearn.model_selection import train_test_split
# Normalize pixel values to between 0 and 1
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train, x_test = (x_train / 255.0), (x_test / 255.0)
#x_train = (x_train / 255.0)

#One hot encode y values for neural network. 
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

#x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.2)

#############################
#Load model wothout classifier/fully connected layers
MobileNet_model = MobileNet(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in MobileNet_model.layers:
	layer.trainable = False
    
MobileNet_model.summary()  #Trainable parameters will be 0

In [ ]:
#Now, let us use features from convolutional network for Model
feature_extractor=MobileNet_model.predict(x_train)

features = feature_extractor.reshape(feature_extractor.shape[0], -1)

X_for_RF = features #This is our X input to Model

In [ ]:
#Send test data through same feature extractor process
X_test_feature = MobileNet_model.predict(x_test)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)



In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=42)
svc.fit(X_for_RF, y_train)

prediction_SVC = svc.predict(X_test_features)
prediction_SVC = le.inverse_transform(prediction_SVC)
print("Accuracy of SVC =",metrics.accuracy_score(test_labels,prediction_SVC))



In [ ]:
from sklearn.metrics import plot_roc_curve
import matplotlib.pyplot as plt
svc_disp = plot_roc_curve(svc, X_test_features, y_test)
ax=plt.gca()
#plt.show()
#rfc_disp = plot_roc_curve(RF_model, X_test_features, y_test,ax = ax,alpha = 0.8)
#svc_disp.plot(ax = ax, alpha=0.8)
plt.show()

In [ ]:
#Confusion Matrix - verify accuracy of each class in SVC
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(test_labels, prediction_SVC)
#print(cm)
sns.heatmap(cm, annot=True)

#Check results on a few select images
n=np.random.randint(0, x_test.shape[0])
img = x_test[n]
plt.imshow(img)
input_img = np.expand_dims(img, axis=0) #Expand dims so the input is (num images, x, y, c)
input_img_feature=MobileNet_model.predict(input_img)
input_img_features=input_img_feature.reshape(input_img_feature.shape[0], -1)
prediction_F1 = svc.predict(input_img_features)[0] 
prediction_F1 = le.inverse_transform([prediction_F1])  #Reverse the label encoder to original name
print("The prediction for this image is: ", prediction_F1)
print("The actual label for this image is: ", test_labels[n])